<center>

## Sevilla - Análisis barrios y precio vivienda según servicios
____

<center>
    
#### Proyecto final del curso "Data Science Coursera Capstone Project"

##### En el siguiente proyecto, trataremos de contestar la siguiente pregunta: ¿Cuáles son los barrios con características similares de Sevilla dependiendo de los servicios que ofrece?

#### Para ello, seguiremos el siguiente proceso:
 -  Importamos librerias que nos sean útiles
 -  Generaremos un Data Frame con los barrios de Sevilla
 -  Incluiremos en el Data Frame la longitud y latitud de cada barrio
 -  Visualizaremos estos resultados
 -  Utilizaremos la API de Foursquare para obtener los datos de los servicios más concurridos en cada barrio
 -  Utilizaremos una forma de agrupación (Clustering: K-Means) para 
 -  Visualizaremos la segmentación

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import csv

#!pip install geocoder
import geocoder # to get longitude and latitude

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # Map plotting library
import numpy as np
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


In [2]:
path = "file:///C:/Users/pablo/Downloads/barrios%20sevilla.xls"

In [3]:
print ("Las primeras cinco filas de nuestra base de datos")
sevilla_data=pd.read_excel(path)
sevilla_data.head()

Las primeras cinco filas de nuestra base de datos


,Código distrito,Distrito,Barrio
0,1,Casco Antiguo,Alfalfa
1,1,Casco Antiguo,Arenal
2,1,Casco Antiguo,Encarnación-Regina
3,1,Casco Antiguo,Feria
4,1,Casco Antiguo,Museo


In [4]:
print ("Las últimas cinco filas de nuestra base de datos")
sevilla_data.tail()

Las últimas cinco filas de nuestra base de datos


,Código distrito,Distrito,Barrio
103,10,Bellavista-La Palmera,Heliópolis
104,10,Bellavista-La Palmera,Pedro Salvador-Las Palmeritas-Guadaira
105,10,Bellavista-La Palmera,Sector Sur-La Palmera-Reina Mercedes
106,11,Los Remedios,Los Remedios
107,11,Los Remedios,Tablada


In [5]:
sevilla_data.shape

(108, 3)

In [6]:
#juntar barrios

In [7]:
# Initialize variables
lat = []
lng = []
lat_lng_coords = None

# Get postcodes from neighbourhoods table
barrios = sevilla_data['Barrio']

# Store latitude and longitude values in lat and lng
for nh in barrios:
    g = geocoder.arcgis('{}, Sevilla, ES'.format(nh))
    lat_lng_coords = g.latlng
    lat.append(lat_lng_coords[0])
    lng.append(lat_lng_coords[1])

In [8]:
sev_data = sevilla_data
sev_data['Latitude'] = lat
sev_data['Longitude'] = lng

In [9]:
sev_data.head()

,Código distrito,Distrito,Barrio,Latitude,Longitude
0,1,Casco Antiguo,Alfalfa,37.39135,-5.99311
1,1,Casco Antiguo,Arenal,37.38648,-5.99751
2,1,Casco Antiguo,Encarnación-Regina,37.39457,-5.99262
3,1,Casco Antiguo,Feria,37.39782,-5.99010
4,1,Casco Antiguo,Museo,37.39166,-5.99956


In [21]:
sev_data.shape

(108, 5)

#### Lets visit Sevilla (on a map)

In [10]:
address = 'Sevilla, Andalucia'
geolocator = Nominatim(user_agent="sevilla_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sevilla are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sevilla are 37.3886303, -5.9953403.


In [18]:
# create map of New York using latitude and longitude values
map_sevilla = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, distrito, barrios in zip(sev_data['Latitude'], sev_data['Longitude'], sev_data['Barrio'], sev_data['Distrito']):
    label = '{}, {}'.format('Distrito', 'Barrio')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sevilla)  
    
map_sevilla

#### Now lets import Data from Foursquare API

In [19]:
CLIENT_ID = 'JSPX5JW3JS0IMMPZFV4RC3OKYMNMNMGUJUETGTDCVD2XEMA2' # your Foursquare ID
CLIENT_SECRET = 'INY1M5ETAYK2FYYBFLCRVYV0M2XLBKMAR2S3C4WGDUFLIJ2H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JSPX5JW3JS0IMMPZFV4RC3OKYMNMNMGUJUETGTDCVD2XEMA2
CLIENT_SECRET:INY1M5ETAYK2FYYBFLCRVYV0M2XLBKMAR2S3C4WGDUFLIJ2H


In [22]:
# Gets the name of the category

def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
explore_df_list=[]

for i, nhood_name in enumerate(sev_data['Barrio']):  
    
    try :
        #Get neighbourhood data
        nhood_name = sev_data.loc[i, 'Barrio']
        nhood_lat = sev_data.loc[i, 'Latitude']
        nhood_lng = sev_data.loc[i, 'Longitude']

        radius = 1000 
        LIMIT = 30 

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nhood_lat, nhood_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flatten JSON

        # Filter the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Rename the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Get the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)    

        # Get the required data
        for i, name in enumerate(nearby['Name']):
            s_list = nearby.loc[i, :].values.tolist()  # Converts the numpy array to a python list
            f_list = [nhood_name, nhood_lat, nhood_lng] + s_list
            explore_df_list.append(f_list)
    
    except Exception as e:
        pass

<ipython-input-28-a6b398dde08e>:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby = json_normalize(results) # Flatten JSON


#### Now we crate a table with all our principal data

In [34]:
explore_sev = pd.DataFrame([item for item in explore_df_list])
explore_sev.columns = ['Barrio', 'Latitud Barrio', 'Longitud Barrio', 'Nombre Avenida', 'Categoria Avenida', 'Latitud Avenida', 'Longitud Avenida']
explore_sev.head()

,Barrio,Latitud Barrio,Longitud Barrio,Nombre Avenida,Categoria Avenida,Latitud Avenida,Longitud Avenida
0,Alfalfa,37.39135,-5.99311,Plaza del Salvador,Plaza,37.390027,-5.993214
1,Alfalfa,37.39135,-5.99311,Perro Viejo,Tapas Restaurant,37.392880,-5.992899
2,Alfalfa,37.39135,-5.99311,Bar el Comercio,Café,37.390772,-5.992341
3,Alfalfa,37.39135,-5.99311,La Libélula,Clothing Store,37.390671,-5.993307
4,Alfalfa,37.39135,-5.99311,Heladería Bolas,Ice Cream Shop,37.389838,-5.992372


In [32]:
explore_sev.shape

(2743, 7)